# 🧪 Lab 2: Use an MCP Server as a Plugin in a Semantic Kernel Agent

In this lab, you'll learn how to **extend a Semantic Kernel agent** by connecting it to an **external MCP server**. MCP (Model Context Protocol) allows agents to invoke external tools, services, or other agents as plugins.

You'll specifically:
- Connect to the **GitHub MCP server** as a tool via `MCPStdioPlugin`
- Use this plugin inside a **Semantic Kernel agent**
- Interact with the agent through a chat interface
- See the agent automatically call functions on the GitHub MCP server to answer questions

This lab showcases how Semantic Kernel can leverage **modular, tool-augmented AI workflows** by treating external MCP servers as powerful extensions to the agent's reasoning capabilities.


In [ ]:
import os

from openai import AsyncAzureOpenAI
from dotenv import load_dotenv

from semantic_kernel.connectors.ai import FunctionChoiceBehavior
from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureChatPromptExecutionSettings,
)
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread

# Setup
load_dotenv(override=True)

The code block below initializes and connects to a GitHub MCP (Model Context Protocol) plugin using the `MCPStdioPlugin` interface.

- `MCPStdioPlugin` is used to run the plugin as a subprocess that communicates via standard input/output (stdio).
- The plugin is a Docker container hosted at `ghcr.io/github/github-mcp-server`.
- The container is run with the `--rm` and `-i` flags:
  - `--rm`: Automatically remove the container when it exits.
  - `-i`: Keep STDIN open to allow interactive communication with the plugin.
- An environment variable `GITHUB_PERSONAL_ACCESS_TOKEN` is passed into the container, which is retrieved from the local environment using `os.getenv(...)`.

Finally, `await github_plugin.connect()` establishes the connection so that the plugin can be used as part of an MCP-enabled agent workflow.

In [ ]:
github_plugin = MCPStdioPlugin(
    name="Github",
    description="Github Plugin",
    command="docker",
    args=[
        "run",
        "-i",
        "--rm",
        "-e",
        "GITHUB_PERSONAL_ACCESS_TOKEN",
        "ghcr.io/github/github-mcp-server",
    ],
    env={"GITHUB_PERSONAL_ACCESS_TOKEN": os.getenv("GITHUB_PERSONAL_ACCESS_TOKEN")},
)

# Start the connection to the MCP plugin
await github_plugin.connect()

Now, let's set up an AI agent that can interact with an external MCP server as a plugin.

We’ll first configure the Azure OpenAI chat completion service, then define an agent by giving it a name, description, and a clear set of instructions. 

Finally, we attach the MCP-based GitHub plugin so the agent can call functions exposed by the plugin to fulfill user requests dynamically.


In [ ]:
# Create the chat completion service
service_id = "azure_openai_chat"
async_openai_client = AsyncAzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_KEY"),
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_version=os.getenv("MODEL_DEPLOYMENT_API_VERSION"),
)
chat_service = AzureChatCompletion(
    service_id=service_id,
    async_client=async_openai_client,
    deployment_name=os.getenv("MODEL_DEPLOYMENT_NAME"),
)
settings = AzureChatPromptExecutionSettings(service_id=service_id)
settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Create the agent with the chat service and plugin
github_agent = ChatCompletionAgent(
    service=chat_service,
    name="GithubAgent",
    description="A chat bot that helps users interact with Github.",
    instructions="""
You are a chat bot. And you help users interact with Github.
You are especially good at answering questions about the Microsoft semantic-kernel project.
You can call functions to get the information you need.
""",
    plugins=[github_plugin],
)

Now let's try it out! 🎉

We're spinning up a simple chat loop where you can talk directly with the agent — powered by Azure OpenAI and enhanced with GitHub superpowers via the MCP plugin.

Let’s see this agent in action! 🤖💬


In [ ]:
# Create the thread
thread: ChatHistoryAgentThread = ChatHistoryAgentThread()

# Run the agent as a chat
while True:
    user_input = input("Enter your message:")
    print(f"\nYou: {user_input}")

    if user_input.lower() in ["exit", "quit"]:
        break

    response = await github_agent.get_response(
        messages=user_input,
        thread=thread,
    )

    assistant_reply = str(response)
    print(f"Bot: {assistant_reply}")

await github_plugin.close()

### 🧪 Try it out!

In this lab, you used the GitHub MCP Server — but the real power comes from the flexibility of the `MCPStdioPlugin` in Semantic Kernel. This component allows you to connect to **any tool that implements the Model Context Protocol (MCP)**.

You can easily swap in other MCP-compatible servers, or even chain multiple plugins together to create powerful tool-augmented agents.

---

🧰 **Explore These MCP Plugin Servers:**

- 🔗 [Official MCP Server Integrations (GitHub)](https://github.com/modelcontextprotocol/servers?tab=readme-ov-file#%EF%B8%8F-official-integrations)  
  A growing list of plugins including GitHub, Weather, Jira, Open Interpreter, and more.

- 📚 [10 Must-Know MCP Servers for Developers (DevShorts)](https://www.devshorts.in/p/ten-must-know-mcp-servers-for-every?utm_source=chatgpt.com)  
  A curated blog post with descriptions, commands, and usage tips.
